In [5]:

Machine Learning Model Dev
Import packages
There are some packages for doing descriptive analytics as follows:

pandas: for data manipulation
numpy: for linear algebra calculation
matplotlib: for data visualization
seaborn: for data manipulation
plotnine: for data manipulation
Note: that there are more than one package used for making a data visualization. The plotnine can be your choice if you are familiar with ggplot2 on R programming. It will create your visualization beautifully

# Dataframe manipulation
import pandas as pd

# Linear algebra
import numpy as np

# Data visualization with plotnine
from plotnine import *
import plotnine

# Data visualization with matplotlib
import matplotlib.pyplot as plt

# Data partitioning
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

# Grid-search
from sklearn.model_selection import GridSearchCV

# Evaluation metrics
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import make_scorer

# XGBoost model
import xgboost as xgb

# Save the model
import joblib
# Ignore warnings
import warnings
warnings.filterwarnings('ignore', category = FutureWarning)
Import data set
After importing the data set into Python, the df_train is now our data frame. The data frame has a lot of functions and methods that will create spesific outputs about the characteristic of data frame. The method of columns will print out all the column names.

Training set
# Import the training set
df_train = pd.read_csv(
    filepath_or_buffer = 'https://raw.githubusercontent.com/dphi-official/Datasets/master/Loan_Data/loan_train.csv',
    usecols = [i for i in range(1, 14)]
)
# Data dimension
print('Data dimension: {} rows and {} columns'.format(len(df_train), len(df_train.columns)))
df_train.head()
Data dimension: 491 rows and 13 columns
Loan_ID	Gender	Married	Dependents	Education	Self_Employed	ApplicantIncome	CoapplicantIncome	LoanAmount	Loan_Amount_Term	Credit_History	Property_Area	Loan_Status
0	LP002305	Female	No	0	Graduate	No	4547	0.0	115.0	360.0	1.0	Semiurban	1
1	LP001715	Male	Yes	3+	Not Graduate	Yes	5703	0.0	130.0	360.0	1.0	Rural	1
2	LP002086	Female	Yes	0	Graduate	No	4333	2451.0	110.0	360.0	1.0	Urban	0
3	LP001136	Male	Yes	0	Not Graduate	Yes	4695	0.0	96.0	NaN	1.0	Urban	1
4	LP002529	Male	Yes	2	Graduate	No	6700	1750.0	230.0	300.0	1.0	Semiurban	1
Testing data
# Import the testing set
df_test = pd.read_csv(
    filepath_or_buffer = 'https://raw.githubusercontent.com/dphi-official/Datasets/master/Loan_Data/loan_test.csv'
)
# Data dimension
print('Data dimension: {} rows and {} columns'.format(len(df_test), len(df_test.columns)))
df_test.head()
Data dimension: 123 rows and 12 columns
Loan_ID	Gender	Married	Dependents	Education	Self_Employed	ApplicantIncome	CoapplicantIncome	LoanAmount	Loan_Amount_Term	Credit_History	Property_Area
0	LP001116	Male	No	0	Not Graduate	No	3748	1668.0	110.0	360.0	1.0	Semiurban
1	LP001488	Male	Yes	3+	Graduate	No	4000	7750.0	290.0	360.0	1.0	Semiurban
2	LP002138	Male	Yes	0	Graduate	No	2625	6250.0	187.0	360.0	1.0	Rural
3	LP002284	Male	No	0	Not Graduate	No	3902	1666.0	109.0	360.0	1.0	Rural
4	LP002328	Male	Yes	0	Not Graduate	No	6096	0.0	218.0	360.0	0.0	Rural
Data preprocessing
Training data
Scale measurement
The method of info will show us the metadata or information about the columns in a data frame. It undirectly specifies the scale measurement of a given columns in a data frame. However, it can be misleading. So, we must modify the scale measurement or column types based on column characteristic.

# Data frame metadata
df_train.info()
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491 entries, 0 to 490
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            491 non-null    object 
 1   Gender             481 non-null    object 
 2   Married            490 non-null    object 
 3   Dependents         482 non-null    object 
 4   Education          491 non-null    object 
 5   Self_Employed      462 non-null    object 
 6   ApplicantIncome    491 non-null    int64  
 7   CoapplicantIncome  491 non-null    float64
 8   LoanAmount         475 non-null    float64
 9   Loan_Amount_Term   478 non-null    float64
 10  Credit_History     448 non-null    float64
 11  Property_Area      491 non-null    object 
 12  Loan_Status        491 non-null    int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 50.0+ KB
# Change column types
df_train = df_train.astype({'Credit_History': object, 'Loan_Status': int})
df_train.select_dtypes(include = ['object']).dtypes
Loan_ID           object
Gender            object
Married           object
Dependents        object
Education         object
Self_Employed     object
Credit_History    object
Property_Area     object
dtype: object
# Summary statistics of categorical columns
for i in df_train.select_dtypes('object').columns:
    print(df_train[i].value_counts(),'\n')
LP002097    1
LP002720    1
LP001673    1
LP002161    1
LP002494    1
           ..
LP002842    1
LP002693    1
LP002961    1
LP002446    1
LP002335    1
Name: Loan_ID, Length: 491, dtype: int64 

Male      393
Female     88
Name: Gender, dtype: int64 

Yes    324
No     166
Name: Married, dtype: int64 

0     276
1      85
2      78
3+     43
Name: Dependents, dtype: int64 

Graduate        388
Not Graduate    103
Name: Education, dtype: int64 

No     398
Yes     64
Name: Self_Employed, dtype: int64 

1.0    380
0.0     68
Name: Credit_History, dtype: int64 

Semiurban    186
Urban        155
Rural        150
Name: Property_Area, dtype: int64 

Handle missing values
# Check missing values
df_train.isna().sum()
Loan_ID               0
Gender               10
Married               1
Dependents            9
Education             0
Self_Employed        29
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           16
Loan_Amount_Term     13
Credit_History       43
Property_Area         0
Loan_Status           0
dtype: int64
Note: Consideration to remove missing values is based on a business logic. The concept of garbage in garbage out applies. Without any relevant domain knowledges of loan problem, the interpolation will lead to the biased result.

Instead of dropping the missing values brutally, we try to inspect the relevant variables in the data in order to suggest the consideration for the next analysis

Dependents
print('Number of missing dependents is about {} rows'.format(df_train['Dependents'].isna().sum()))
Number of missing dependents is about 9 rows
# Replace missing valuess with "0"
df_train['Dependents'].fillna(value = '0', inplace = True) 
Self_Employed
print('Number of missing Self_Employed is about {} rows'.format(df_train['Self_Employed'].isna().sum()))
Number of missing Self_Employed is about 29 rows
# Replace missing values with "No"
df_train['Self_Employed'].fillna(value = 'No', inplace = True) 
Loan_Amount_Term
df_train[['Loan_Amount_Term', 'Loan_Status']].groupby('Loan_Status').describe()
Loan_Amount_Term
count	mean	std	min	25%	50%	75%	max
Loan_Status								
0	143.0	341.790210	73.018891	36.0	360.0	360.0	360.0	480.0
1	335.0	341.086567	64.320411	12.0	360.0	360.0	360.0	480.0
print('Percentile 20th: {}'.format(df_train['Loan_Amount_Term'].quantile(q = 0.2)))
Percentile 20th: 360.0
# Replace missing values with "360"
df_train['Loan_Amount_Term'].fillna(value = 360, inplace = True)
Credit_History
# Cross tabulation of credit history and loan status
df_cred_hist = pd.crosstab(df_train['Credit_History'], df_train['Loan_Status'], margins = True).reset_index()
# Remove index name
df_cred_hist.columns.name = None
# Remove last row for total column attribute
df_cred_hist = df_cred_hist.drop([len(df_cred_hist) - 1], axis = 0)
df_cred_hist.rename(columns = {'Credit_History':'Credit History', 0:'No', 1:'Yes'}, inplace = True)
df_cred_hist
Credit History	No	Yes	All
0	0	62	6	68
1	1	74	306	380
# Slice the data frame based on loan status
pos_cred_hist0 = df_train[(df_train['Credit_History'].isna()) & (df_train['Loan_Status'] == 0)]
pos_cred_hist1 = df_train[(df_train['Credit_History'].isna()) & (df_train['Loan_Status'] == 1)]
print('Number of rows with Loan_Status is No but Credit_History is NaN  : {}'.format(len(pos_cred_hist0)))
print('Number of rows with Loan_Status is Yes but Credit_History is NaN : {}'.format(len(pos_cred_hist1)))
Number of rows with Loan_Status is No but Credit_History is NaN  : 12
Number of rows with Loan_Status is Yes but Credit_History is NaN : 31
# Replace the missing values with a specific condition
credit_loan = zip(df_train['Credit_History'], df_train['Loan_Status'])
df_train['Credit_History'] = [
                                0.0 if np.isnan(credit) and status == 0 else
                                1.0 if np.isnan(credit) and status == 1 else
                                credit for credit, status in credit_loan
                             ]
Gender and Loan Amount
# Drop missing values
df_train.dropna(axis = 0, how = 'any', inplace = True)
# Check missing value
df_train.isna().sum()
Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64
Testing data
Scale measurement
# Data frame metadata
df_test.info()
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            123 non-null    object 
 1   Gender             120 non-null    object 
 2   Married            121 non-null    object 
 3   Dependents         117 non-null    object 
 4   Education          123 non-null    object 
 5   Self_Employed      120 non-null    object 
 6   ApplicantIncome    123 non-null    int64  
 7   CoapplicantIncome  123 non-null    float64
 8   LoanAmount         117 non-null    float64
 9   Loan_Amount_Term   122 non-null    float64
 10  Credit_History     116 non-null    float64
 11  Property_Area      123 non-null    object 
dtypes: float64(4), int64(1), object(7)
memory usage: 11.7+ KB
# Change column types
df_test = df_test.astype({'Credit_History': object})
df_test.select_dtypes(include = ['object']).dtypes
Loan_ID           object
Gender            object
Married           object
Dependents        object
Education         object
Self_Employed     object
Credit_History    object
Property_Area     object
dtype: object
# Summary statistics of categorical columns
for i in df_test.select_dtypes('object').columns:
    print(df_test[i].value_counts(),'\n')
LP002862    1
LP001350    1
LP001492    1
LP002128    1
LP001868    1
           ..
LP002933    1
LP001109    1
LP002984    1
LP002328    1
LP002515    1
Name: Loan_ID, Length: 123, dtype: int64 

Male      96
Female    24
Name: Gender, dtype: int64 

Yes    74
No     47
Name: Married, dtype: int64 

0     69
2     23
1     17
3+     8
Name: Dependents, dtype: int64 

Graduate        92
Not Graduate    31
Name: Education, dtype: int64 

No     102
Yes     18
Name: Self_Employed, dtype: int64 

1.0    95
0.0    21
Name: Credit_History, dtype: int64 

Urban        47
Semiurban    47
Rural        29
Name: Property_Area, dtype: int64 

Handle missing values
# Check missing values
df_test.isna().sum()
Loan_ID              0
Gender               3
Married              2
Dependents           6
Education            0
Self_Employed        3
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           6
Loan_Amount_Term     1
Credit_History       7
Property_Area        0
dtype: int64
Dependents
print('Number of missing values in Dependents is about {} rows'.format(df_test['Dependents'].isna().sum()))
Number of missing values in Dependents is about 6 rows
# Replace missing values with "0"
df_test['Dependents'].fillna(value = '0', inplace = True)
Self_Employed
print('Number of missing values in Self_Employed is about {} rows'.format(df_test['Self_Employed'].isna().sum()))
Number of missing values in Self_Employed is about 3 rows
# Replace missing values with "No"
df_test['Self_Employed'].fillna(value = 'No', inplace = True) 
Loan_Amount_Term
# Replace missing values with "360"
df_test['Loan_Amount_Term'].fillna(value = 360, inplace = True)
Gender, Married, LoanAmount and Credit_History
# Drop missing values
df_test.dropna(axis = 0, how = 'any', inplace = True)
# Check missing values
df_test.isna().sum()
Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
dtype: int64
Explanatory data analysis
The composition of default and not default customers
# Data aggregation between default and not default customers
df_viz_1 = df_train.groupby(['Loan_Status'])['Loan_ID'].count().reset_index(name = 'Total')
# Map the loan status
df_viz_1['Loan_Status'] = df_viz_1['Loan_Status'].map(
    {
        0: 'Not default',
        1: 'Default'
    }
)
# Show the data
df_viz_1
Loan_Status	Total
0	Not default	134
1	Default	330
# Figure size
plt.figure(figsize = (6.4,4.8))

# Customize colors and other settings
colors = ['#80797c','#981220']

# Explode 1st slice
explode = (0.1, 0)

# Create a pie chart
plt.pie(
    x = 'Total',
    labels = 'Loan_Status',
    data = df_viz_1,
    explode = explode,
    colors = colors,
    autopct = '%1.1f%%',
    shadow = False,
    startangle = 140
)

# Title and axis
plt.title('Number of customers by loan status', fontsize = 18)
plt.axis('equal')
plt.show()

The composition of loan status by the dependents
# Data aggregation between loan status and dependents
df_viz_2 = df_train.groupby(['Loan_Status', 'Dependents'])['Loan_ID'].count().reset_index(name = 'Total')
# Map the loan status
df_viz_2['Loan_Status'] = df_viz_2['Loan_Status'].map(
    {
        0: 'Not default',
        1: 'Default'
    }
)
# Show the data
df_viz_2
Loan_Status	Dependents	Total
0	Not default	0	77
1	Not default	1	30
2	Not default	2	13
3	Not default	3+	14
4	Default	0	191
5	Default	1	52
6	Default	2	62
7	Default	3+	25
plotnine.options.figure_size = (8, 4.8)
(
    ggplot(
        data = df_viz_2
    )+
    geom_bar(
        aes(
            x = 'Dependents',
            y = 'Total',
            fill = 'Loan_Status'
        ),
        stat = 'identity',
        position = 'fill',
        width = 0.5
    )+
    labs(
        title = 'The composition of loan status by the dependents',
        fill = 'Loan status'
    )+
    xlab(
        'Dependents'
    )+
    ylab(
        'Frequency'
    )+
    scale_x_discrete(
        limits = ['0', '1', '2', '3+']
    )+
    scale_fill_manual(
        values = ['#981220','#80797c'],
        labels = ['Default', 'Not Default']
    )+
    theme_minimal()
)

<ggplot: (80233892994)>
The composition of default customer by the educations
# Data aggregation between loan status and dependents
df_viz_3 = df_train.groupby(['Loan_Status', 'Education'])['Loan_ID'].count().reset_index(name = 'Total')
# Map the loan status
df_viz_3['Loan_Status'] = df_viz_3['Loan_Status'].map(
    {
        0: 'Not default',
        1: 'Default'
    }
)
# Show the data
df_viz_3
Loan_Status	Education	Total
0	Not default	Graduate	101
1	Not default	Not Graduate	33
2	Default	Graduate	266
3	Default	Not Graduate	64
plotnine.options.figure_size = (8, 4.8)
(
    ggplot(
        data = df_viz_3
    )+
    geom_bar(
        aes(
            x = 'Education',
            y = 'Total',
            fill = 'Loan_Status'
        ),
        stat = 'identity',
        position = 'fill',
        width = 0.5
    )+
    labs(
        title = 'The composition of loan status by the education',
        fill = 'Loan status'
    )+
    xlab(
        'Educations'
    )+
    ylab(
        'Frequency'
    )+
    scale_x_discrete(
        limits = ['Graduate', 'Not Graduate']
    )+
    scale_fill_manual(
        values = ['#981220','#80797c'],
        labels = ['Default', 'Not Default']
    )+
    theme_minimal()
)

<ggplot: (80234761253)>
The distribution of applicant incomes by loan status
# Slice the columns
df_viz_4 = df_train[['ApplicantIncome', 'Loan_Status']].reset_index(drop = True)
# Map the loan status
df_viz_4['Loan_Status'] = df_viz_4['Loan_Status'].map(
    {
        0: 'Not default',
        1: 'Default'
    }
)
# Show the data
df_viz_4.head()
ApplicantIncome	Loan_Status
0	4547	Default
1	5703	Default
2	4333	Not default
3	4695	Default
4	6700	Default
plotnine.options.figure_size = (8, 4.8)
(
    ggplot(
        data = df_viz_4
    )+
    geom_density(
        aes(
            x = 'ApplicantIncome',
            fill = 'Loan_Status'
        ),
        color = 'white',
        alpha = 0.85
    )+
    labs(
        title = 'The distribution of applicant incomes by loan status'
    )+
    scale_fill_manual(
        name = 'Loan Status',
        values = ['#981220','#80797c'],
        labels = ['Default', 'Not Default']
    )+
    xlab(
        'Applicant income'
    )+
    ylab(
        'Density'
    )+
    theme_minimal()
)

<ggplot: (80234767177)>
The distribution of loan amount by loan status
# Slice the columns
df_viz_5 = df_train[['LoanAmount', 'Loan_Status']].reset_index(drop = True)
# Map the loan status
df_viz_5['Loan_Status'] = df_viz_5['Loan_Status'].map(
    {
        0: 'Not default',
        1: 'Default'
    }
)
# Show the data
df_viz_5.head()
LoanAmount	Loan_Status
0	115.0	Default
1	130.0	Default
2	110.0	Not default
3	96.0	Default
4	230.0	Default
plotnine.options.figure_size = (8, 4.8)
(
    ggplot(
        data = df_viz_5
    )+
    geom_density(
        aes(
            x = 'LoanAmount',
            fill = 'Loan_Status'
        ),
        color = 'white',
        alpha = 0.85
    )+
    labs(
        title = 'The distribution of loan amount by loan status'
    )+
    scale_fill_manual(
        name = 'Loan Status',
        values = ['#981220','#80797c'],
        labels = ['Default', 'Not Default']
    )+
    xlab(
        'Loan amount'
    )+
    ylab(
        'Density'
    )+
    theme_minimal()
)

<ggplot: (80233503016)>
One-hot encoder
# Add new column of Loan_Status with 999 in testing data
df_test['Loan_Status'] = 999
# Concat the training and testing data
df_concat = pd.concat(objs = [df_train , df_test], axis = 0)
# Drop the column of Loan_ID
df_concat.drop(columns = ['Loan_ID'], inplace = True)
# Categorical columns
cols_obj_train = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Credit_History', 'Property_Area']
print(cols_obj_train)
['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Credit_History', 'Property_Area']
# One-hot encoding
df_concat = pd.get_dummies(data = df_concat, columns = cols_obj_train, drop_first = True)
print('Dimension data: {} rows and {} columns'.format(len(df_concat), len(df_concat.columns)))
df_concat.head()
Dimension data: 570 rows and 15 columns
ApplicantIncome	CoapplicantIncome	LoanAmount	Loan_Amount_Term	Loan_Status	Gender_Male	Married_Yes	Dependents_1	Dependents_2	Dependents_3+	Education_Not Graduate	Self_Employed_Yes	Credit_History_1.0	Property_Area_Semiurban	Property_Area_Urban
0	4547	0.0	115.0	360.0	1	0	0	0	0	0	0	0	1	1	0
1	5703	0.0	130.0	360.0	1	1	1	0	0	1	1	1	1	0	0
2	4333	2451.0	110.0	360.0	0	0	1	0	0	0	0	0	1	0	1
3	4695	0.0	96.0	360.0	1	1	1	0	0	0	1	1	1	0	1
4	6700	1750.0	230.0	300.0	1	1	1	0	1	0	0	0	1	1	0
Data partitioning
# Unique values of Loan_Status
df_concat['Loan_Status'].value_counts()
1      330
0      134
999    106
Name: Loan_Status, dtype: int64
# Training set
df_train = df_concat[df_concat['Loan_Status'].isin([0, 1])].reset_index(drop = True)
print('Dimension data: {} rows and {} columns'.format(len(df_train), len(df_train.columns)))
df_train.head()
Dimension data: 464 rows and 15 columns
ApplicantIncome	CoapplicantIncome	LoanAmount	Loan_Amount_Term	Loan_Status	Gender_Male	Married_Yes	Dependents_1	Dependents_2	Dependents_3+	Education_Not Graduate	Self_Employed_Yes	Credit_History_1.0	Property_Area_Semiurban	Property_Area_Urban
0	4547	0.0	115.0	360.0	1	0	0	0	0	0	0	0	1	1	0
1	5703	0.0	130.0	360.0	1	1	1	0	0	1	1	1	1	0	0
2	4333	2451.0	110.0	360.0	0	0	1	0	0	0	0	0	1	0	1
3	4695	0.0	96.0	360.0	1	1	1	0	0	0	1	1	1	0	1
4	6700	1750.0	230.0	300.0	1	1	1	0	1	0	0	0	1	1	0
# Testing set
df_test = df_concat[df_concat['Loan_Status'].isin([999])].reset_index(drop = True)
print('Data dimension: {} rows and {} columns'.format(len(df_test), len(df_test.columns)))
df_test.head()
Data dimension: 106 rows and 15 columns
ApplicantIncome	CoapplicantIncome	LoanAmount	Loan_Amount_Term	Loan_Status	Gender_Male	Married_Yes	Dependents_1	Dependents_2	Dependents_3+	Education_Not Graduate	Self_Employed_Yes	Credit_History_1.0	Property_Area_Semiurban	Property_Area_Urban
0	3748	1668.0	110.0	360.0	999	1	0	0	0	0	1	0	1	1	0
1	4000	7750.0	290.0	360.0	999	1	1	0	0	1	0	0	1	1	0
2	2625	6250.0	187.0	360.0	999	1	1	0	0	0	0	0	1	0	0
3	3902	1666.0	109.0	360.0	999	1	0	0	0	0	1	0	1	0	0
4	6096	0.0	218.0	360.0	999	1	1	0	0	0	1	0	0	0	0
# Data partitioning >>> training set into training and validation
df_train_final = df_train.reset_index(drop = True)
X = df_train_final[df_train_final.columns[~df_train_final.columns.isin(['Loan_Status'])]]
y = df_train_final['Loan_Status']

# Training = 70% and validation = 30%
X_train, X_val, y_train, y_val = train_test_split(X , y, test_size = 0.3, random_state = 42)
print('Data dimension of training set   :', X_train.shape)
print('Data dimension of validation set :', X_val.shape)

# Testing set
X_test = df_test[df_test.columns[~df_test.columns.isin(['Loan_Status'])]]
print('Data dimension of testing set    :', X_test.shape)
Data dimension of training set   : (324, 14)
Data dimension of validation set : (140, 14)
Data dimension of testing set    : (106, 14)
Machine learning model development
# XGBoost model
xgb_model = xgb.XGBClassifier(
    objective = 'binary:logistic',
    use_label_encoder = False
)
# Define parameter range 
params = {
    'eta': np.arange(0.1, 0.26, 0.05),
    'min_child_weight': np.arange(1, 5, 0.5).tolist(),
    'gamma': [5],
    'subsample': np.arange(0.5, 1.0, 0.11).tolist(),
    'colsample_bytree': np.arange(0.5, 1.0, 0.11).tolist()
}
# Make a scorer from a performance metric or loss function
scorers = {
    'f1_score': make_scorer(f1_score),
    'precision_score': make_scorer(precision_score),
    'recall_score': make_scorer(recall_score),
    'accuracy_score': make_scorer(accuracy_score)
}
# k-fold cross validation
skf = KFold(n_splits = 10, shuffle = True)
# Set up the grid search CV
grid = GridSearchCV(
    estimator = xgb_model,
    param_grid = params,
    scoring = scorers,
    n_jobs = -1,
    cv = skf.split(X_train, np.array(y_train)),
    refit = 'accuracy_score'
)
# Fit the model
grid.fit(X = X_train, y = y_train)
[15:57:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
GridSearchCV(cv=<generator object _BaseKFold.split at 0x0000012AE43D40B0>,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=n...
             param_grid={'colsample_bytree': [0.5, 0.61, 0.72, 0.83, 0.94],
                         'eta': array([0.1 , 0.15, 0.2 , 0.25]), 'gamma': [5],
                         'min_child_weight': [1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0,
                                              4.5],
                         'subsample': [0.5, 0.61, 0.72, 0.83, 0.94]},
             refit='accuracy_score',
             scoring={'accuracy_score': make_scorer(accuracy_score),
                      'f1_score': make_scorer(f1_score),
                      'precision_score': make_scorer(precision_score),
                      'recall_score': make_scorer(recall_score)})
# Best parameters
grid.best_params_
{'colsample_bytree': 0.72,
 'eta': 0.15000000000000002,
 'gamma': 5,
 'min_child_weight': 1.5,
 'subsample': 0.61}
# Create a prediction of training 
predicted = grid.predict(X_val)
# Model evaluation - training data
accuracy_baseline = accuracy_score(predicted, np.array(y_val))
recall_baseline = recall_score(predicted, np.array(y_val))
precision_baseline = precision_score(predicted, np.array(y_val))
f1_baseline = f1_score(predicted, np.array(y_val))

print('Accuracy for baseline   :{}'.format(round(accuracy_baseline, 5)))
print('Recall for baseline     :{}'.format(round(recall_baseline, 5)))
print('Precision for baseline  :{}'.format(round(precision_baseline, 5)))
print('F1 Score for baseline   :{}'.format(round(f1_baseline, 5)))
Accuracy for baseline   :0.87143
Recall for baseline     :0.84615
Precision for baseline  :1.0
F1 Score for baseline   :0.91667
Store the ML model
# Store the model into a pickle file
filename = '../bin/xgboostModel.pkl'
joblib.dump(grid.best_estimator_, filename)
['../bin/xgboostModel.pkl']